In [50]:
# Imports

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import wandb

wandb.login()

True

In [51]:
# Device

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [52]:
# Define Hyperparameters

config = dict (
    input_size=784,
    epochs=1, 
    classes=10,
    kernels=[50,50],
    batch_size=64,
    learning_rate=0.005,
    dataset="MNIST",
    architecture="ANN"
)

In [53]:
# Model Pipeline

def model_pipeline(hyperparameters):

    with wandb.init(project="wandb_test", config=hyperparameters):
        
        config = wandb.config

        train_loader, test_loader = make_data(config)
        
        model, criterion, optimizer = make_model(config)
        print(model)

        train(model, train_loader, criterion, optimizer, config)

        test(model, test_loader)

    return model




In [54]:
# Make Data

def make_data(config):

    train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=config.batch_size, shuffle=True)

    test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=config.batch_size, shuffle=True)
 
    return train_loader, test_loader

In [55]:
# Make Model

class NeuralNetwork(nn.Module):

    def __init__(self, input_size, kernels, classes):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, kernels[0])
        self.layer2 = nn.Linear(kernels[0], kernels[1])

    def forward(self, x):
        output = F.relu(self.layer1(x))
        output = self.layer2(output)
        return output



def make_model(config):
    
    model = NeuralNetwork(input_size=config.input_size, kernels=config.kernels, classes=config.classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    return model, criterion, optimizer


In [56]:
# Train Model

def train(model, train_loader, criterion, optimizer, config):

    wandb.watch(model, criterion, log="all", log_freq=5)

    for epoch in range(config.epochs):
        for batch_index, (data, targets) in enumerate(train_loader):

            loss = train_batch(data, targets, model, optimizer, criterion)

            if((batch_index + 1) % 10) == 0:
                train_log(loss, epoch, batch_index)




def train_batch(data, targets, model, optimizer, criterion):

    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape(data.shape[0], -1)

    scores = model(data)
    loss = criterion(scores, targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    return loss

    

In [57]:
def train_log(loss, epoch, batch_index):
    loss = float(loss)

    wandb.log({"epoch:": epoch, "loss": loss}, step=batch_index)

In [58]:
def test(model, test_loader):
    model.eval()

    with torch.no_grad():
        correct, total = 0, 0
        for data, target in test_loader:
            data = data.to(device=device)
            target = target.to(device=device)

            data = data.reshape(data.shape[0], -1)

            score = model(data)
            _, predicted = torch.max(score.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        wandb.log({"test_accuracy": correct / total})

  
        

In [59]:
model_pipeline(config)

NeuralNetwork(
  (layer1): Linear(in_features=784, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
)


_runtime,26
_timestamp,1631446626
_step,929
epoch:,0
loss,0.05207
test_accuracy,0.9516


_runtime,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇█
_timestamp,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇█
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch:,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▄▃▂▂▃▃▂▃▂▂▃▂▁▂▂▂▁▁▁▂▂▁▂▁▂▁▂▂▃▂▂▂▁▂▁▂▁▁▁
test_accuracy,▁


NeuralNetwork(
  (layer1): Linear(in_features=784, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
)